# Strategy Serialization with BoFire

## Imports

In [ ]:
from pydantic import TypeAdapter

import bofire.strategies.api as stategies
from bofire.benchmarks.multi import DTLZ2
from bofire.benchmarks.single import Himmelblau
from bofire.data_models.acquisition_functions.api import qLogNEI
from bofire.data_models.domain.api import Domain, Outputs
from bofire.data_models.kernels.api import RBFKernel, ScaleKernel
from bofire.data_models.strategies.api import AnyStrategy
from bofire.data_models.strategies.api import QnehviStrategy as QnehviStrategyDataModel
from bofire.data_models.strategies.api import RandomStrategy as RandomStrategyDataModel
from bofire.data_models.strategies.api import SoboStrategy as SoboStrategyDataModel
from bofire.data_models.surrogates.api import BotorchSurrogates, SingleTaskGPSurrogate
from bofire.surrogates.diagnostics import CvResults2CrossValidationValues
from bofire.surrogates.trainable import TrainableSurrogate

## Single Objective Problem Setup

In [ ]:
benchmark = Himmelblau()
samples = benchmark.domain.inputs.sample(n=10)

# this is the training data
experiments = benchmark.f(samples, return_complete=True)

# this are the pending candidates
pending_candidates = benchmark.domain.inputs.sample(2)

## Random Strategy

The random strategy and other strategies that just inherit from `Strategy` and not `PredictiveStrategy` are special as they do not need defined output features in the domain and they do not need a call to `tell` before the `ask`. Furthermore they online provide input features in the candidates and no predictions for output features.

In [ ]:
# setup the data model
domain = Domain(inputs=benchmark.domain.inputs)
strategy_data = RandomStrategyDataModel(domain=domain)

# we generate the json spec
jspec = strategy_data.model_dump_json()

jspec

'{"type":"RandomStrategy","domain":{"type":"Domain","inputs":{"type":"Inputs","features":[{"type":"ContinuousInput","key":"x_1","unit":null,"bounds":[-6.0,6.0],"local_relative_bounds":null,"stepsize":null},{"type":"ContinuousInput","key":"x_2","unit":null,"bounds":[-6.0,6.0],"local_relative_bounds":null,"stepsize":null}]},"outputs":{"type":"Outputs","features":[]},"constraints":{"type":"Constraints","constraints":[]}},"seed":null,"fallback_sampling_method":"UNIFORM","n_burnin":1000,"n_thinning":32,"num_base_samples":null,"max_iters":1000}'

In [ ]:
# load it
strategy_data = TypeAdapter(AnyStrategy).validate_json(jspec)

# map it
strategy = stategies.map(strategy_data)

# ask it
df_candidates = strategy.ask(candidate_count=5)

# transform to spec
candidates = strategy.to_candidates(df_candidates)

candidates

[Candidate(inputValues={'x_1': InputValue(value='-2.081380869279547'), 'x_2': InputValue(value='-3.4925266804803234')}, outputValues=None),
 Candidate(inputValues={'x_1': InputValue(value='-0.9753297614125493'), 'x_2': InputValue(value='2.8605625476069516')}, outputValues=None),
 Candidate(inputValues={'x_1': InputValue(value='0.5673745273160868'), 'x_2': InputValue(value='-2.741953904997784')}, outputValues=None),
 Candidate(inputValues={'x_1': InputValue(value='-1.0933887850890756'), 'x_2': InputValue(value='-5.904583395012207')}, outputValues=None),
 Candidate(inputValues={'x_1': InputValue(value='3.7567124306933'), 'x_2': InputValue(value='-4.341022073090965')}, outputValues=None)]

## SOBO Strategy

Setup the strategies data model.

In [ ]:
# setup the data model
strategy_data = SoboStrategyDataModel(
    domain=benchmark.domain, acquisition_function=qLogNEI()
)

# we generate the json spec
jspec = strategy_data.model_dump_json()

jspec

'{"type":"SoboStrategy","domain":{"type":"Domain","inputs":{"type":"Inputs","features":[{"type":"ContinuousInput","key":"x_1","unit":null,"bounds":[-6.0,6.0],"local_relative_bounds":null,"stepsize":null},{"type":"ContinuousInput","key":"x_2","unit":null,"bounds":[-6.0,6.0],"local_relative_bounds":null,"stepsize":null}]},"outputs":{"type":"Outputs","features":[{"type":"ContinuousOutput","key":"y","unit":null,"objective":{"type":"MinimizeObjective","w":1.0,"bounds":[0.0,1.0]}}]},"constraints":{"type":"Constraints","constraints":[]}},"seed":null,"num_restarts":8,"num_raw_samples":1024,"maxiter":2000,"batch_limit":8,"descriptor_method":"EXHAUSTIVE","categorical_method":"EXHAUSTIVE","discrete_method":"EXHAUSTIVE","surrogate_specs":{"surrogates":[{"hyperconfig":{"type":"SingleTaskGPHyperconfig","hyperstrategy":"FactorialStrategy","inputs":{"type":"Inputs","features":[{"type":"CategoricalInput","key":"kernel","categories":["rbf","matern_1.5","matern_2.5"],"allowed":[true,true,true]},{"type":"

As SOBO is a predictive strategy, training data has to be provided before candidated can be requested. 

In [ ]:
# load it
strategy_data = TypeAdapter(AnyStrategy).validate_json(jspec)

# map it
strategy = stategies.map(strategy_data)

# tell it the pending candidates if present
if pending_candidates is not None:
    strategy.add_candidates(pending_candidates)

# tell it
strategy.tell(experiments=experiments)

# ask it
df_candidates = strategy.ask(candidate_count=2)

# transform to spec
candidates = strategy.to_candidates(df_candidates)

candidates

[Candidate(inputValues={'x_1': InputValue(value='2.09984061271394'), 'x_2': InputValue(value='-0.26379286778242955')}, outputValues={'y': OutputValue(predictedValue='67.46964918807359', standardDeviation=106.18730170073752, objective=-67.46964918807359)}),
 Candidate(inputValues={'x_1': InputValue(value='-6.0'), 'x_2': InputValue(value='1.0692194677683151')}, outputValues={'y': OutputValue(predictedValue='144.17329044678652', standardDeviation=143.42723730176186, objective=-144.17329044678652)})]

We can also save the trained models of the strategy, for more info look at the `model_serial.ipynb` notebook. It could be that the `dumps` command fails here. But this is already fixed in the main branch of the `linear_operator` package, and if not yet, it should be available in main soon.

In [ ]:
jsurrogate_spec = strategy_data.surrogate_specs.surrogates[0].json()
dump = strategy.surrogates.surrogates[0].dumps()

## MOBO Strategy

As example for a multiobjective strategy we are using here the Qnehvi stratey. Related strategies would be Qparego, MultiplicativeSobo etc. To use it, we have to first generate a multiobjective domain.

In [ ]:
benchmark = DTLZ2(dim=6)
samples = benchmark.domain.inputs.sample(n=20)
experiments = benchmark.f(samples, return_complete=True)
pending_candidates = benchmark.domain.inputs.sample(2)

Now the strategy spec is setup. Note that we can define there exactly which model to use.

In [ ]:
# setup the data model
strategy_data = QnehviStrategyDataModel(
    domain=benchmark.domain,
    surrogate_specs=BotorchSurrogates(
        surrogates=[
            SingleTaskGPSurrogate(
                inputs=benchmark.domain.inputs,
                outputs=Outputs(features=[benchmark.domain.outputs[0]]),
                kernel=ScaleKernel(base_kernel=RBFKernel(ard=False)),
            )
        ]
    ),
)

# we generate the json spec
jspec = strategy_data.model_dump_json()

jspec

'{"type":"QnehviStrategy","domain":{"type":"Domain","inputs":{"type":"Inputs","features":[{"type":"ContinuousInput","key":"x_0","unit":null,"bounds":[0.0,1.0],"local_relative_bounds":null,"stepsize":null},{"type":"ContinuousInput","key":"x_1","unit":null,"bounds":[0.0,1.0],"local_relative_bounds":null,"stepsize":null},{"type":"ContinuousInput","key":"x_2","unit":null,"bounds":[0.0,1.0],"local_relative_bounds":null,"stepsize":null},{"type":"ContinuousInput","key":"x_3","unit":null,"bounds":[0.0,1.0],"local_relative_bounds":null,"stepsize":null},{"type":"ContinuousInput","key":"x_4","unit":null,"bounds":[0.0,1.0],"local_relative_bounds":null,"stepsize":null},{"type":"ContinuousInput","key":"x_5","unit":null,"bounds":[0.0,1.0],"local_relative_bounds":null,"stepsize":null}]},"outputs":{"type":"Outputs","features":[{"type":"ContinuousOutput","key":"f_0","unit":null,"objective":{"type":"MinimizeObjective","w":1.0,"bounds":[0.0,1.0]}},{"type":"ContinuousOutput","key":"f_1","unit":null,"object

Generate the candidates.

In [ ]:
# load it
strategy_data = TypeAdapter(AnyStrategy).validate_json(jspec)

# map it
strategy = stategies.map(strategy_data)

# tell it the pending candidates if available
if pending_candidates is not None:
    strategy.add_candidates(pending_candidates)

# tell it
strategy.tell(experiments=experiments)

# ask it
df_candidates = strategy.ask(candidate_count=1)

# transform to spec
candidates = strategy.to_candidates(df_candidates)

candidates

/Users/radical-rhys/miniconda/lib/python3.12/site-packages/botorch/acquisition/multi_objective/monte_carlo.py:321: NumericsWarning: qNoisyExpectedHypervolumeImprovement has known numerical issues that lead to suboptimal optimization performance. It is strongly recommended to simply replace

	 qNoisyExpectedHypervolumeImprovement 	 --> 	 qLogNoisyExpectedHypervolumeImprovement 

instead, which fixes the issues and has the same API. See https://arxiv.org/abs/2310.20708 for details.
  legacy_ei_numerics_warning(legacy_name=type(self).__name__)


[Candidate(inputValues={'x_0': InputValue(value='0.007946289641626523'), 'x_1': InputValue(value='0.0'), 'x_2': InputValue(value='1.0'), 'x_3': InputValue(value='0.04043525745077388'), 'x_4': InputValue(value='0.40632043549036534'), 'x_5': InputValue(value='0.49832520994282303')}, outputValues={'f_0': OutputValue(predictedValue='0.7372714053103276', standardDeviation=0.23158425112625597, objective=-0.7372714053103276), 'f_1': OutputValue(predictedValue='0.5197504060265992', standardDeviation=0.30183832347670936, objective=-0.5197504060265992)})]

To fill the model info section accordingly, the following snippet has to be executed for every surrogate, incldung saving the actual models.

In [ ]:
from typing import Literal

from pydantic import BaseModel


class TestMethod(BaseModel):
    type: str


class CrossValidation(TestMethod):
    type: Literal["CrossValidation"] = "CrossValidation"
    foldCount: int


for i in range(len(strategy_data.surrogate_specs.surrogates)):
    surrogate_data = strategy.surrogate_specs.surrogates[i]
    surrogate = strategy.surrogates.surrogates[i]
    # get the spec
    jsurrogate_spec = surrogate_data.json()
    # get the dump
    dump = surrogate.dumps()
    # do the cross validation, only if we have a trainable model under the hood
    if isinstance(surrogate, TrainableSurrogate):
        cv_train, cv_test, _ = surrogate.cross_validate(strategy.experiments, folds=5)
        # transform the bofire objects to the backend objects
        testMethod = CrossValidation(foldCount=5)
        cvResultsTrain = CvResults2CrossValidationValues(cv_train)
        cvResultsTest = CvResults2CrossValidationValues(cv_test)
        metricsTrain = {
            surrogate.outputs[0].key: cv_train.get_metrics(combine_folds=False)
            .describe()
            .loc["mean"]
            .to_dict()
        }
        metricsTest = {
            surrogate.outputs[0].key: cv_test.get_metrics(combine_folds=True)
            .describe()
            .loc["mean"]
            .to_dict()
        }
        # save to backend
        # - jsurrogate_spec
        # - dump
        # - testMethod
        # - cvResultsTrain
        # - cvResultsTest
        # - metricsTrain
        # - metricsTest